# 목표:
- VGG16 모델의 사전 학습된 w, b를 통한 feature을 가지고
- Flatten층부터 Classification 층을 다시 그려서
- test 데이터를 Flickrapi를 통해 가지고 온 이미지를 prediction한다.

In [1]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\fra

In [2]:
# 모든 이미지를 1/255로 스케일을 조정합니다
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

base_dir = './datasets/'

train_dir = os.path.join(base_dir, 'train')
# file_list = os.listdir(train_dir)
# print("file_list: {}".format(file_list))
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
    # 타깃 디렉터리
    train_dir,
    # 모든 이미지를 224 × 224 크기로 바꿉니다
    target_size=(224, 224),
    batch_size=10,
    # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
    class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
    # 타깃 디렉터리
    validation_dir,
    # 모든 이미지를 224 × 224 크기로 바꿉니다
    target_size=(224, 224),
    batch_size=10,
    # binary_crossentropy 손실을 사용하기 때문에 이진 레이블이 필요합니다
    class_mode='binary')

Found 147 images belonging to 2 classes.
Found 27 images belonging to 2 classes.


In [3]:
for data_batch, labels_batch in train_generator:
    print('Batch size:', data_batch.shape)
    print('Label size:', labels_batch.shape)
    break

Batch size: (10, 224, 224, 3)
Label size: (10,)


In [4]:
# This code cell shows how to utilize VGG16 model by combining Dense layer at the end
from keras.layers import Input, Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model

from keras.applications.vgg16 import VGG16
from keras import layers
from keras import models

VGGNet = VGG16()
VGGNet.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_______________________________________________________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
_________________________________________________________________
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 56, 56, 256)       590080    
_________________________________________________________________
block3_conv3 (Conv2D)        (None, 56, 56, 256)       590080    
_________________________________________________________________
block3_pool (MaxPooling2D)   (None, 28, 28, 256)       0         
_________________________________________________________________
block4_conv1 (Conv2D)        (None, 28, 28, 512)       1180160   
_________________________________________________________________
block4_conv2 (Conv2D)        (None, 28, 28, 512)       2359808   
_________________________________________________________________
block4_conv3 (Conv2D)        (None, 28, 28, 512)       2359808   
_________________________________________________________________
block4_pool (MaxPooling2D)   (None, 14, 14, 512)       0         
_________________________________________________________________
block5_conv1 (Conv2D)        (None, 14, 14, 512)       2359808   
_________________________________________________________________
block5_conv2 (Conv2D)        (None, 14, 14, 512)       2359808   
_________________________________________________________________
block5_conv3 (Conv2D)        (None, 14, 14, 512)       2359808   
_________________________________________________________________
block5_pool (MaxPooling2D)   (None, 7, 7, 512)         0         
_________________________________________________________________

## 여기까지가 feature를 뽑는 단계


## 여기부터가 아래에서 다시 그려줄 것이다.
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              102764544 
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 1000)              4097000   
_________________________________________________________________

## 여기까지가 feature를 가지고 Classification하는 층

Total params: 138,357,544
Trainable params: 138,357,544
Non-trainable params: 0
_________________________________________________________________

In [5]:
# We will not update VGG pre-trained model, only added Dense layers will be trained from the scratch

for layer in VGGNet.layers:
    layer.trainable = False # vgg16모델의 w, b를 업데이트(학습)하지 않겠다.

vgg_maxpool5 = VGGNet.get_layer('block5_pool').output

Feature_Flatten = Flatten()(vgg_maxpool5)   # Flatten층 그래프
dense = Dense(10, name='dense', activation='relu')(Feature_Flatten)   #Dense층 그래프
predictions = Dense(1, activation='sigmoid')(dense)

New_VGGmodel = Model(inputs=VGGNet.input, outputs=predictions)
New_VGGmodel.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

New_VGGmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [6]:
history = New_VGGmodel.fit_generator(train_generator,
                                    # validation_data=validation_generator,
                                    # validation_steps=10,
                                    steps_per_epoch=5,
                                    epochs=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
5/5 [==============================] - 3s 509ms/step - loss: 0.8203 - accuracy: 0.5400
Epoch 2/5
5/5 [==============================] - 1s 204ms/step - loss: 1.0341 - accuracy: 0.5745
Epoch 3/5
5/5 [==============================] - 0s 79ms/step - loss: 0.7103 - accuracy: 0.6000
Epoch 4/5
5/5 [==============================] - 1s 110ms/step - loss: 0.6160 - accuracy: 0.6000
Epoch 5/5
5/5 [==============================] - 1s 102ms/step - loss: 0.5495 - accuracy: 0.6800


In [7]:
print(validation_generator.class_indices)

{'Cats': 0, 'Tigers': 1}


In [8]:
from flickrapi import FlickrAPI

FLICKER_KEY = ''
FLICKER_SECRET = ''

flickr = FlickrAPI(FLICKER_KEY, FLICKER_SECRET, format='parsed-json')
extras='url_s'
cats = flickr.photos.search(text='tiger', per_page=5, extras=extras)
photos = cats['photos']

In [9]:
# import the necessary packages
import numpy as np
import urllib
import cv2

# METHOD #1: OpenCV, NumPy, and urllib
# url을 이미지로 변경하는 함
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urllib.request.urlopen(url)
    # print(resp)
    image = np.asarray(bytearray(resp.read()), dtype='uint8')
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    
    return image

In [10]:
image_original_url = photos['photo'][0]['url_s']

image_temp = url_to_image(image_original_url)
cv2.imshow('image_temp', image_temp)
cv2.waitKey(0)

-1

In [11]:
resized_image = cv2.resize(image_temp, (224,224))
print(resized_image.shape)

(224, 224, 3)


In [12]:
resized_image = np.expand_dims(resized_image, axis=0)
print(resized_image.shape)

(1, 224, 224, 3)


In [13]:
result = New_VGGmodel.predict(resized_image)

In [14]:
result

array([[0.9285635]], dtype=float32)